# Pré processamento dos dados

- [Link dos dados](https://ckan.pbh.gov.br/dataset/ec3efaac-0ca6-4846-9e32-0ffff2d76dbb/resource/4189b28e-7592-474b-8160-e39731315c27/download/20250401_atividade_economica.csv)
- baixar o csv e colocar na pasta `./data`

In [1]:
import pandas as pd
import re
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
file_path = 'data/20250401_atividade_economica.csv'
df = pd.read_csv(file_path, sep=';')

In [3]:
df.head()

,ID_ATIV_ECON_ESTABELECIMENTO,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL,CNAE,DATA_INICIO_ATIVIDADE,NATUREZA_JURIDICA,PORTE_EMPRESA,AREA_UTILIZADA,IND_SIMPLES,IND_MEI,...,FORMA_ATUACAO,DESC_LOGRADOURO,NOME_LOGRADOURO,NUMERO_IMOVEL,COMPLEMENTO,NOME_BAIRRO,NOME,NOME_FANTASIA,CNPJ,GEOMETRIA
0,1006,9521500.0,REPARAÇÃO E MANUTENÇÃO DE EQUIPAMENTOS ELETROE...,"3314707, 4322302, 4649404, 4651601, 4669999, 4...",19-04-1994,SOCIEDADE EMPRESÁRIA LIMITADA,EMPRESA DE PEQUENO PORTE,407.0,S,N,...,ESTABELECIMENTO FIXO,RUA,NIQUELINA,874,"GALPAO,",SANTA EFIGENIA,ORLANDO SERVICE LTDA,NaN,97502553000184,POINT (613854.46 7797085.18)
1,1007,4755502.0,COMERCIO VAREJISTA DE ARTIGOS DE ARMARINHO,"4755502, 8599604",12-04-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,20.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DOS GUAJAJARAS,910,SALA 612,CENTRO,TARGET LTDA,NaN,70950258000134,POINT (610658.10 7796519.70)
2,1008,4520001.0,SERVIÇOS DE MANUTENÇÃO E REPARAÇÃO MECÂNICA DE...,"4520001, 4520003, 4530703",16-11-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,260.0,S,N,...,ESTABELECIMENTO FIXO,RUA,CONSELHEIRO LAFAIETE,284,NaN,SAGRADA FAMILIA,HUDSON CAR LTDA,HUDSON CAR,731235000153,POINT (612523.34 7798131.15)
3,1009,4616800.0,REPRESENTANTES COMERCIAIS E AGENTES DO COMÉRCI...,4616800,16-11-1992,SOCIEDADE EMPRESÁRIA LIMITADA,EMPRESA DE PEQUENO PORTE,1300.0,S,N,...,ESTABELECIMENTO FIXO,RUA,DOS TIMBIRAS,2352,SALA 01,LOURDES,SILHUETA REPRESENTACAO DE ROUPAS LTDA,GARMENT,68519362000127,POINT (610394.12 7796444.35)
4,1010,5822101.0,EDIÇÃO INTEGRADA À IMPRESSÃO DE JORNAIS DIÁRIOS,5822101,01-06-1993,SOCIEDADE EMPRESÁRIA LIMITADA,MICROEMPRESA - ME,30.0,N,N,...,ESTABELECIMENTO FIXO,RUA,SAO FELICIANO,73,NaN,CAICARAS,JORNAL CAICARAS LTDA,NaN,71078968000189,POINT (607875.47 7799877.16)


In [4]:
df['DESC_LOGRADOURO'].value_counts()

DESC_LOGRADOURO
RUA    447131
AVE     79646
BEC      2648
PCA      2053
ROD      1854
ALA      1007
EST       296
TRV       214
VIA        18
VDP         8
LRG         3
TRE         2
Name: count, dtype: int64

In [5]:
NOVO_LOGRADOURO = {'RUA': 'RUA', 'AVE': 'AV', 'PCA': 'PC', 'BEC': 'BC', 'ROD': 'RD', 'ALA': 'AL',
                   'EST': 'ESTRADA', 'TRV': 'TRV', 'VIA': 'VIA', 'VDP': 'V', 'LRG': 'LRG', 'TRE': 'TRE'}

# no dataset filtrado nao foi encontrado LRG e nem TRE, mative o mesmo

In [6]:
df['DESC_LOGRADOURO'] = df['DESC_LOGRADOURO'].map(NOVO_LOGRADOURO)

In [7]:
print (df.columns)

Index(['ID_ATIV_ECON_ESTABELECIMENTO', 'CNAE_PRINCIPAL',
       'DESCRICAO_CNAE_PRINCIPAL', 'CNAE', 'DATA_INICIO_ATIVIDADE',
       'NATUREZA_JURIDICA', 'PORTE_EMPRESA', 'AREA_UTILIZADA', 'IND_SIMPLES',
       'IND_MEI', 'IND_POSSUI_ALVARA', 'TIPO_UNIDADE', 'FORMA_ATUACAO',
       'DESC_LOGRADOURO', 'NOME_LOGRADOURO', 'NUMERO_IMOVEL', 'COMPLEMENTO',
       'NOME_BAIRRO', 'NOME', 'NOME_FANTASIA', 'CNPJ', 'GEOMETRIA'],
      dtype='object')


## Filtragem por bares e restaurantes

In [8]:
words = ['bar', 'bares', 'restaurante', 'restaurantes']

Constrói a expressão regex que da match em qualquer uma das palavras, se ela aparecer como uma palavra separada por espaços ou pontuações das outras:

In [9]:
pattern = r'\b(?:' + '|'.join(re.escape(word) for word in words) + r')\b'
pattern

'\\b(?:bar|bares|restaurante|restaurantes)\\b'

In [10]:
filtered_table = df[['CNAE_PRINCIPAL', 'DESCRICAO_CNAE_PRINCIPAL']].drop_duplicates()
filtered_table = filtered_table[filtered_table['DESCRICAO_CNAE_PRINCIPAL'].str.contains(pattern, case=False, na=False)]

for value in filtered_table['DESCRICAO_CNAE_PRINCIPAL']:
    print(value)
filtered_table

RESTAURANTES E SIMILARES
BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS EM SERVIR BEBIDAS, SEM ENTRETENIMENTO
BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS EM SERVIR BEBIDAS, COM ENTRETENIMENTO 


,CNAE_PRINCIPAL,DESCRICAO_CNAE_PRINCIPAL
17,5611201.0,RESTAURANTES E SIMILARES
679,5611204.0,BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS...
3079,5611205.0,BARES E OUTROS ESTABELECIMENTOS ESPECIALIZADOS...


In [11]:
CNAES = [int(cnae) for cnae in filtered_table['CNAE_PRINCIPAL']]
CNAES

[5611201, 5611204, 5611205]

In [12]:
filtered_df = df[df['CNAE_PRINCIPAL'].isin(CNAES)].copy()

filtered_df['CNAE_PRINCIPAL'].value_counts()

CNAE_PRINCIPAL
5611201.0    8550
5611204.0    3570
5611205.0    1505
Name: count, dtype: int64

## Formatação do Endereço

In [13]:
filtered_df['ENDERECO_FORMATADO'] = (
  filtered_df['DESC_LOGRADOURO'] + ' ' +
  filtered_df['NOME_LOGRADOURO'] + ', ' +
  filtered_df['NUMERO_IMOVEL'].astype(str)
)

filtered_df[['ENDERECO_FORMATADO']].head()

,ENDERECO_FORMATADO
17,"RUA DESEMBARGADOR REIS ALVES, 90"
19,"RUA LUIZ PONGELUPE, 290"
29,"AV RESSACA, 118"
172,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135"
363,"AV CRISTIANO MACHADO, 1950"


## Tabela Final

In [14]:
filtered_table = filtered_df[['DATA_INICIO_ATIVIDADE', 'IND_POSSUI_ALVARA', 'ENDERECO_FORMATADO', 'NOME_FANTASIA']].drop_duplicates()

filtered_table = filtered_table[filtered_table['NOME_FANTASIA'].notna()]

filtered_table['ID'] = range(1, filtered_table.shape[0]+1)

filtered_table.head()

,DATA_INICIO_ATIVIDADE,IND_POSSUI_ALVARA,ENDERECO_FORMATADO,NOME_FANTASIA,ID
19,15-10-1993,NÃO,"RUA LUIZ PONGELUPE, 290",PIZZARIA E CHURRASCARIA VARANDA,1
29,02-05-1994,NÃO,"AV RESSACA, 118",RABBIT BURGER,2
172,24-11-1993,SIM,"RUA DESEMBARGADOR RIBEIRO DA LUZ, 135",TATU REI DO ANGU A BAHIANA,3
363,01-10-1993,SIM,"AV CRISTIANO MACHADO, 1950",ESPACO DA FEIJOADA,4
662,16-09-2011,SIM,"RUA DOS OTONI, 222",RESTAURANTE NOVO SABOR,5


In [15]:
filtered_table.to_csv('data/dados.csv', index=False)

In [16]:
# teste do api_geo
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable
import time

USER_AGENT = "tp1algoritmos_ufmg"
INPUT_CSV_PATH = "data/dados.csv"

try:
    geolocator = Nominatim(user_agent=USER_AGENT)
except Exception as e:
    print(f"Erro ao inicializar o geolocator Nominatim: {e}")
    print("Verifique sua conexão com a internet ou se há restrições de firewall.")
    exit()

def geocode_single_address(address_str, city_context="BELO HORIZONTE", state_context="MG"):
    """
    Tenta geocodificar um único endereço.
    Adiciona contexto de cidade/estado se não parecer estar presente.
    """
    full_address = address_str
    if city_context.lower() not in address_str.lower():
        full_address += f", {city_context}"
    if state_context.lower() not in address_str.lower() and state_context not in full_address: # Evitar duplicar MG se já tiver BH, MG
        full_address += f", {state_context}"

    #print(f"Tentando geocodificar: \"{full_address}\"")
    try:
        # Timeout de 10 segundos para a requisição
        location = geolocator.geocode(full_address, timeout=10)
        if location:
            return location.latitude, location.longitude, location.address
        else:
            print(f"Endereço não encontrado por Nominatim: {full_address}")
            return None, None, None
    except GeocoderTimedOut:
        print(f"Timeout ao tentar geocodificar: {full_address}. O serviço pode estar ocupado.")
        return None, None, None
    except GeocoderUnavailable:
        print(f"Serviço de geocodificação indisponível: {full_address}. Tente novamente mais tarde.")
        return None, None, None
    except Exception as e:
        print(f"Ocorreu um erro inesperado durante a geocodificação de '{full_address}': {e}")
        return None, None, None

In [17]:
dados_geoloc = []
dados_with_error = []
for id, endereco in tqdm(filtered_table[['ID', 'ENDERECO_FORMATADO']].to_numpy()):
    latitude, longitude, EnderecoNominatim = geocode_single_address(endereco)
    if EnderecoNominatim == None: 
        dados_with_error.append([id, EnderecoNominatim])
    dados_geoloc.append([id, latitude, longitude, EnderecoNominatim])

  1%|          | 86/8657 [01:59<3:16:49,  1.38s/it]

Endereço não encontrado por Nominatim: AV ANTONIO HENRIQUE ALVES, 551, BELO HORIZONTE, MG


  1%|          | 102/8657 [02:24<4:50:17,  2.04s/it]

Endereço não encontrado por Nominatim: RUA BENJAMIM QUADROS, 266, BELO HORIZONTE, MG


  1%|          | 106/8657 [02:30<3:38:52,  1.54s/it]

...

Ocorreu um erro inesperado durante a geocodificação de 'RUA SILVA ALVARENGA, 614, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA DOS ADVOGADOS, 300, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'AV OLEGARIO MACIEL, 742, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA PARAIBA, 805, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA ILACIR PEREIRA LIMA, 95, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'AV DOS ANDRADAS, 367, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'AV ALFREDO CAMARATE, 539, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA 

 96%|█████████▌| 8280/8657 [1:37:56<00:02, 162.96it/s]

Ocorreu um erro inesperado durante a geocodificação de 'AV AMAZONAS, 152, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA MAFRA, 735, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA PRINCIPAL, 461, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA CONDOR, 90, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA GRANITO, 127, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA NOSSA SENHORA DE COPACABANA, 331, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA ZILA SILVEIRA, 251, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA LICINIO PORTO, 154, 

100%|██████████| 8657/8657 [1:37:57<00:00,  1.47it/s] 

Ocorreu um erro inesperado durante a geocodificação de 'RUA FERNANDES TOURINHO, 441, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA LUPERCIO PAIXAO, 180, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA ALBERTO CINTRA, 44, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA DOMINGOS ROCHA, 119, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'AV AMINTAS JACQUES DE MORAES, 175, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'RUA SALINAS, 1489, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação de 'AV GUARAPARI, 538, BELO HORIZONTE, MG': [Errno 11001] getaddrinfo failed
Ocorreu um erro inesperado durante a geocodificação d

In [18]:
dados_geoloc

[[1,
  -19.7993229,
  -43.9760286,
  'Rua Luiz Pongellupe, Vila SESC, Jardim dos Comerciários, Venda Nova, Belo Horizonte, Região Geográfica Imediata de Belo Horizonte, Região Metropolitana de Belo Horizonte, Região Geográfica Intermediária de Belo Horizonte, Minas Gerais, Região Sudeste, 31640-060, Brasil'],
 [2,
  -19.921143,
  -43.9860821,
  'Avenida Ressaca, Coração Eucarístico, Regional Noroeste, Belo Horizonte, Região Geográfica Imediata de Belo Horizonte, Região Metropolitana de Belo Horizonte, Região Geográfica Intermediária de Belo Horizonte, Minas Gerais, Região Sudeste, 30535-610, Brasil'],
 [3,
  -19.9732009,
  -44.0141918,
  'Rua Desembargador Ribeiro da Luz, Barreiro de Baixo, Barreiro, Belo Horizonte, Região Geográfica Imediata de Belo Horizonte, Região Metropolitana de Belo Horizonte, Região Geográfica Intermediária de Belo Horizonte, Minas Gerais, Região Sudeste, 30640-020, Brasil'],
 [4,
  -19.8903695,
  -43.9282191,
  'Feira dos Produtores, 1950, Avenida Cristiano Ma

In [19]:
import pickle
import os

def salvar_pkl(objeto, path_completo):
    """
    Salva um objeto Python em formato .pkl no caminho completo especificado.
    
    Args:
        objeto: objeto Python a ser salvo.
        path_completo (str): caminho completo do arquivo, incluindo o nome e a extensão .pkl
    """
    os.makedirs(os.path.dirname(path_completo), exist_ok=True)
    
    with open(path_completo, "wb") as f:
        pickle.dump(objeto, f)
    print(f"Arquivo salvo em: {path_completo}")

def ler_pkl(path_completo):
    """
    Lê um arquivo .pkl a partir do caminho completo especificado.
    
    Args:
        path_completo (str): caminho completo do arquivo, incluindo o nome e a extensão .pkl
    
    Returns:
        objeto Python carregado do arquivo .pkl
    """
    with open(path_completo, "rb") as f:
        return pickle.load(f)

In [ ]:
# salvar_pkl(dados_geoloc, './temp/dado_geoloc_sucesso.pkl')

Arquivo salvo em: ./temp/dado_geoloc_sucesso.pkl


In [27]:
dados_geoloc = ler_pkl( './temp/dado_geoloc_sucesso.pkl')

In [28]:
id_executados = [id for id, lat,_,_ in dados_geoloc if  lat is not None]
novos_dados_geoloc = []

for id, endereco in tqdm(filtered_table[['ID', 'ENDERECO_FORMATADO']].to_numpy()):
    if id not in id_executados: 
        latitude, longitude, EnderecoNominatim = geocode_single_address(endereco)
        novos_dados_geoloc.append([id, latitude, longitude, EnderecoNominatim])

  1%|          | 86/8657 [00:02<04:10, 34.16it/s]

Endereço não encontrado por Nominatim: AV ANTONIO HENRIQUE ALVES, 551, BELO HORIZONTE, MG


  1%|          | 102/8657 [00:03<05:31, 25.79it/s]

Endereço não encontrado por Nominatim: RUA BENJAMIM QUADROS, 266, BELO HORIZONTE, MG


  1%|          | 106/8657 [00:05<08:27, 16.85it/s]

...

100%|█████████▉| 8626/8657 [1:22:37<00:32,  1.06s/it]

Endereço não encontrado por Nominatim: PC TEJO, 12, BELO HORIZONTE, MG


100%|█████████▉| 8632/8657 [1:22:45<00:33,  1.33s/it]

Endereço não encontrado por Nominatim: RUA ADRIANO CHAVES E MATOS, 486, BELO HORIZONTE, MG


100%|██████████| 8657/8657 [1:23:14<00:00,  1.73it/s]


In [29]:
full_dados_geoloc = [row for row in  (novos_dados_geoloc + dados_geoloc) if row[1] is not None]
len(full_dados_geoloc)

8282

In [30]:
salvar_pkl(full_dados_geoloc, './temp/dado_geoloc_sucesso.pkl')

Arquivo salvo em: ./temp/dado_geoloc_sucesso.pkl


In [ ]:
# --- Pegar um endereço de exemplo para testar ---
if 'ENDERECO_FORMATADO' in filtered_table.columns:
    endereco_exemplo = filtered_table['ENDERECO_FORMATADO'].iloc[85]
    print(f"\n--- Testando geocodificação para o endereço de exemplo ---")
    print(f"Endereço original do CSV: \"{endereco_exemplo}\"")

    latitude, longitude, EnderecoNominatim = geocode_single_address(endereco_exemplo)

    if latitude is not None and longitude is not None:
        print("\n--- Resultado da Geocodificação ---")
        print(f"Endereço Original: {endereco_exemplo}")
        print(f"Endereço Retornado pelo Nominatim: {EnderecoNominatim}")
        print(f"Latitude: {latitude}")
        print(f"Longitude: {longitude}")
    else:
        print(f"\nNão foi possível geocodificar o endereço de exemplo: \"{endereco_exemplo}\"")
        print('erro --- (tipo)', EnderecoNominatim)
else:
    if df.empty:
        print("O DataFrame está vazio. Não há endereços para testar.")
    else:
        print("Erro: A coluna 'ENDERECO_FORMATADO' não foi encontrada no CSV.")

print("\n--- Fim do teste de geocodificação para um endereço ---")